In [58]:
from glob import glob
from dateutil import parser
from typing import Union, Optional
import uproot as ur
import numpy as np
import json, zhh
version = 'v1'
REPO_ROOT = '/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH'
DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/Preselection/{version}'
ILD_VERSION = 'ILD_l5_o1_v02'
PROD_NAME = '500-TDR_ws'

In [8]:
metafile = glob(f'{DATA_ROOT}/htcondor_jobs*.json')[0]
with open(metafile) as file:
    meta = json.load(file)

In [11]:
jobs = meta['jobs']
n_tot = len(jobs.keys())
n_done = 0
for branch in jobs:
    if n_done == 0:
        print(jobs[branch])
    
    if jobs[branch]['status'] == 'finished':
        n_done += 1
        
print(f'Done {n_done}/{n_tot}')

{'job_id': 'dummy_job_id', 'branches': [0], 'status': 'finished', 'code': 0, 'error': None, 'extra': {'log': '/nfs/dust/ilc/user/bliewert/zhh/Preselection/v1/stdall_0To1.txt'}}
Done 88/88


In [87]:
def evaluate(branch:Union[int,str]):
    branch = int(branch)
    
    with open(f'{DATA_ROOT}/{branch}_Source.txt') as file:
        src_path = file.read().strip()
        process = src_path.split(f'/{PROD_NAME}/')[1]
        process = process.split(f'/{ILD_VERSION}/')[0]
        
    with open(f'{DATA_ROOT}/stdall_{branch}To{branch+1}.txt') as file:
        signals = ['start time    :', 'end time      :', 'job exit code :']
        values = ['', '', '']
        lsig = len(signals)
        
        for line in file.readlines():
            for i in range(lsig):
                if line.startswith(signals[i]):
                    values[i] = line.split(f'{signals[i]} ')[1].strip()
                    
        for i in [0, 1]:
            if values[i] != '':
                if ' (' in values[i]:
                    values[i] = values[i].split(' (')[0]
                
                values[i] = float(parser.parse(values[i]).timestamp())
            
    return [process, src_path, values[0], values[1], values[1] - values[0], int(values[2])]

evaluate(0)

['2f_Z_bhabhaNg',
 '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250126.P2f_z_bhabhang.eL.pL.n001.d_dstm_10602_0.slcio',
 1721334662.805879,
 1721334803.618228,
 140.81234884262085,
 0]

In [105]:
results = np.empty(0, dtype=[
    ('process', 'U32'),
    ('src', 'U255'),
    ('tStart', 'f'),
    ('tEnd', 'f'),
    ('tDuration', 'f'),
    ('exitCode', 'i')])

for branch in jobs:
    if jobs[branch]['status'] == 'finished':
        ev = evaluate(branch)
        print(ev)
        np.append(results, [ev])

['2f_Z_bhabhaNg', '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250127.P2f_z_bhabhang.eL.pR.n001.d_dstm_10609_0.slcio', 1721334662.633811, 1721334700.75669, 38.12287902832031, 0]


TypeError: The DType <class 'numpy.dtype[void]'> could not be promoted by <class 'numpy.dtype[str_]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[void]'>, <class 'numpy.dtype[str_]'>)

In [106]:
l = '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250127.P2f_z_bhabhang.eL.pR.n001.d_dstm_10609_0.slcio'

In [107]:
len(l)

199

In [108]:
b = ['2f_Z_bhabhaNg', '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250127.P2f_z_bhabhang.eL.pR.n001.d_dstm_10609_0.slcio', 1721334662.633811, 1721334700.75669, 38.12287902832031, 0]

In [109]:
b

['2f_Z_bhabhaNg',
 '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250127.P2f_z_bhabhang.eL.pR.n001.d_dstm_10609_0.slcio',
 1721334662.633811,
 1721334700.75669,
 38.12287902832031,
 0]

In [116]:
results.dtype

dtype([('process', '<U32'), ('src', '<U255'), ('tStart', '<f4'), ('tEnd', '<f4'), ('tDuration', '<f4'), ('exitCode', '<i4')])

In [115]:
np.array([b], dtype=results.dtype)

ValueError: could not convert string to float: '2f_Z_bhabhaNg'

In [110]:
len(b[0])

13

In [112]:
np.append(results, np.array(b))

TypeError: The DType <class 'numpy.dtype[void]'> could not be promoted by <class 'numpy.dtype[str_]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[void]'>, <class 'numpy.dtype[str_]'>)

In [94]:
b is float

False